In [47]:
import pandas as pd
import os,gc
from dotenv import load_dotenv
load_dotenv()

# Example: load 4 CSVs
df1 = pd.read_csv("dwlr_realtime_data.csv")
df2 = pd.read_csv("groundwater_quality_data.csv")
df3 = pd.read_csv("groundwater_trends_2015_2023.csv")
df4 = pd.read_csv("ingres_assessment_units_2023.csv")





In [ ]:
pinecone_api_key = os.environ["PINECONE_API_KEY"]
pinecone_index= os.environ["INDEX"]
gemini_api_key = os.environ["GEMINI_API_KEY"]
print(pinecone_api_key, pinecone_index, gemini_api_key)


pcsk_5GVyAe_3n3unEk9auouRXymiJEYnxJXgSxP3J4BGam9JY7Bp4XQD9FcPCxtiKxLakMBWFd


In [50]:
import os, gc   # ✅ import both together

# Remove API keys from local environment
os.environ.pop("PINECONE_API_KEY", None)
os.environ.pop("GEMINI_API_KEY", None)

# Force garbage collection to release memory
gc.collect()

print("✅ Cleared local memory and API keys (Pinecone cloud index is unaffected)")


✅ Cleared local memory and API keys (Pinecone cloud index is unaffected)


In [3]:
def df_to_text(df, name):
    docs = []
    for _, row in df.iterrows():
        text = f"{name} Record: " + " | ".join([f"{col}: {row[col]}" for col in df.columns if pd.notna(row[col])])
        docs.append(text)
    return docs

docs1 = df_to_text(df1, "Realtime")
docs2 = df_to_text(df2, "Quality")
docs3 = df_to_text(df3, "Trend")
docs4 = df_to_text(df4, "Assessment")

all_docs = docs1 + docs2 + docs3 + docs4


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = splitter.create_documents(all_docs)


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#vectorstore = Chroma.from_documents(documents, embeddings, persist_directory="./chroma_db")
#vectorstore.persist()


In [13]:
print(embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [21]:
from langchain_community.vectorstores import Pinecone as PineconeVectorStore

vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=embeddings,
    index_name=pinecone_index,
)

ValueError: Index 'sih' not found in your Pinecone project. Did you mean one of the following indexes: test, pdf-qa-index